In [1]:
rm(list = ls())
library(tidyverse)
library(readr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients1002.csv") df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients1002.csv")
df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients1003.csv")
#  df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients0807.csv")
#df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients0812.csv")
#  df = read_csv("https://www.city.adachi.tokyo.jp/documents/47358/131200_city_adachi_covid19_patients0814_2.csv")
df = df %>% mutate(g = str_replace_all(備考, "(2020/|し公表)" , ""))


Warning message:
“Missing column names filled in: 'X13' [13]”
Parsed with column specification:
cols(
  `№` = col_double(),
  市区町村コード = col_double(),
  都道府県 = col_character(),
  市区町村名 = col_character(),
  診断日 = col_date(format = ""),
  居住地 = col_character(),
  年代 = col_character(),
  性別 = col_character(),
  渡航歴 = col_character(),
  症状 = col_character(),
  `回復・死亡` = col_character(),
  備考 = col_character(),
  X13 = col_logical()
)



In [3]:
df = df %>% mutate(年代 = str_replace(年代 , "３０代" , "30代") %>%
                        str_replace( "５０代" , "50代") %>%
                       str_replace( "２０代" , "20代") %>%
                       str_replace("8０代", "80代") %>% 
                       str_replace("４０代"  , "40代"))
start_date  =as.Date("2020-1-1")
end_date  = as.Date("2020-12-31")
#df = df  %>% mutate(is_na_biko = str_count(備考, "集団") == 0)
df = df  %>% mutate(is_na_biko = is.na(g))
df1 = df %>% filter(診断日 >= start_date & 診断日 <= end_date) 
df1 %>% dim

[1] 1154   15

In [7]:
#df1$symptom %>% unique
df1$症状 %>% str_split("、") %>% flatten %>% as.character %>%  unique

df1 = df1 %>% mutate(symptom = 　str_replace_all(症状, "なし" , "無症状") %>% 
                       str_replace_all("障害|異常", "")%>%
                     str_replace_all("発熱嗅覚", "発熱・嗅覚")%>%
                     str_replace_all("嗅覚味覚", "嗅覚・味覚")) %>%
  mutate(symptom = ifelse(str_count(symptom , "急性呼吸器") > 0 , "急性呼吸器症状" , symptom))

[1] "咳"                             "発熱"                          
 [3] "関節痛"                         "無症状"                        
 [5] "味覚・嗅覚障害"                 "発熱　咳"                      
 [7] "呼吸苦"                         "食欲不振"                      
 [9] "頭痛"                           "痰"                            
[11] "重篤な肺炎"                     "急性呼吸器症候群"              
[13] "嗅覚障害"                       "下痢"                          
[15] "倦怠感"                         "心肺停止"                      
[17] "咳以外の急性呼吸器症状"         "なし"                          
[19] "味覚障害"                       "発熱　"                        
[21] "肺炎"                           "発熱　咳以外の急性呼吸器症状　"
[23] "発熱　肺炎"                     "鼻汁"                          
[25] "嗅覚・味覚障害"                 "発熱　咳　"                    
[27] "咽頭痛"                         "咽頭通"                        
[29] "発熱　咽頭痛"                   ""                              
[31] "嗅覚・味覚異常"                 "嗅覚味覚障害"                  
[33] "心不全"                         "発熱嗅覚・味覚障害"            
[35] "肺炎像"                         "意識障害"                      
[37] "吐気/嘔吐"                      "咽頭違和感"                    
[39] "腹痛"                           "急性呼吸器症状"                
[41] "全身倦怠感"                     "嘔気"                          
[43] "くしゃみ"                       "鼻汁鼻閉"                      
[45] "鼻閉"                           "嘔気/嘔吐"                     
[47] "発熱　肺炎像"                   "鼻閉感"                        
[49] "咳　下痢"                       "咳,発熱"

In [24]:
symptoms = df1$symptom %>% str_split("、| |・|・|,|/|\\s") %>% flatten %>% as.character %>% 
　　  str_replace_all("なし" , "無症状") %>% unique %>% setdiff("")

In [25]:
df2 = df1
for (s in symptoms){
 df2 = df2 %>% mutate(!!s := ( str_count(symptom , s) >0) )
}

# 検定手法について

http://cse.naro.affrc.go.jp/takezaawa/r-tips/r/66.html

https://sudori.info/stat/stat_contigency_table.html

https://ja.wikipedia.org/wiki/%E3%82%AB%E3%82%A4%E4%BA%8C%E4%B9%97%E6%A4%9C%E5%AE%9A
の独立性検定の項目によると
二項分布を正規分布で近似できないようなサンプル数が少ない場合はG検定が良いとのこと
あるいは二項検定かフィッシャーの正確確率検定
https://data-science.gr.jp/implementation/ist_r_fisher_exact_probability_test.html



In [31]:
?chisq.test

chisq.test {stats},R Documentation
x,a numeric vector or matrix. x and y can also both be factors.
y,"a numeric vector; ignored if x is a matrix. If x is a factor, y should be a factor of the same length."
correct,"a logical indicating whether to apply continuity correction when computing the test statistic for 2 by 2 tables: one half is subtracted from all |O - E| differences; however, the correction will not be bigger than the differences themselves. No correction is done if simulate.p.value = TRUE."
p,a vector of probabilities of the same length of x. An error is given if any entry of p is negative.
rescale.p,"a logical scalar; if TRUE then p is rescaled (if necessary) to sum to 1. If rescale.p is FALSE, and p does not sum to 1, an error is given."
simulate.p.value,a logical indicating whether to compute p-values by Monte Carlo simulation.
B,an integer specifying the number of replicates used in the Monte Carlo test.
statistic,the value the chi-squared test statistic.
parameter,"the degrees of freedom of the approximate chi-squared distribution of the test statistic, NA if the p-value is computed by Monte Carlo simulation."
p.value,the p-value for the test.


## chi-squared test

In [16]:
chisq.test(df2$発熱 , df2$咽頭痛)


	Pearson's Chi-squared test with Yates' continuity correction

data:  df2$発熱 and df2$咽頭痛
X-squared = 5.1489, df = 1, p-value = 0.02326


In [28]:
chisq.test(df2$発熱 , df2$味覚)


	Pearson's Chi-squared test with Yates' continuity correction

data:  df2$発熱 and df2$味覚
X-squared = 0.01981, df = 1, p-value = 0.8881


## fisher's exact test

In [35]:
xtabs(~発熱+味覚 , df2)

       味覚
発熱  FALSE TRUE
  FALSE   400   83
  TRUE    559  112

In [36]:
fisher.test(xtabs(~発熱+味覚 , df2))


	Fisher's Exact Test for Count Data

data:  xtabs(~発熱 + 味覚, df2)
p-value = 0.8735
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.6993167 1.3369356
sample estimates:
odds ratio 
 0.9655896 


In [29]:
chisq.test(df2$発熱 , df2$嗅覚)


	Pearson's Chi-squared test with Yates' continuity correction

data:  df2$発熱 and df2$嗅覚
X-squared = 0.2932, df = 1, p-value = 0.5882


In [10]:
sf_all = data.frame()
#sf_all_biko = data.frame()
for (s in symptoms) {
  if (F) {
    s = symptoms[1]
  }
  if (T) {
      run_sf = df1 %>% mutate(run_sym =str_count(symptom  ,  s) >0 ) %>% 
    group_by(年代, 性別 , is_na_biko ,run_sym) %>% summarize(n = n()) %>% ungroup %>% mutate(symptom = s)
      
  } else {
    run_sf = df1 %>% mutate(run_sym =str_count(symptom  ,  s) >0 ) %>% 
      group_by(年代, run_sym) %>% summarize(n = n()) %>% ungroup %>% mutate(symptom = s)
  }
  sf_all = bind_rows(sf_all , run_sf)
}

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代', '性別', 'is_na_biko' (override with `.groups` argument)

`summarise()` regrouping output by '年代',

In [ ]:
sf_all %>% head
if (T) {
  df1b = df1
  for (s in symptoms) {
      df1b = df1b %>% mutate(!!s := pmin(1 ,  str_count(symptom  , 　s)  ))
  }
  #https://stats.stackexchange.com/questions/408564/relationship-independence-between-2-binary-varibales
  #https://oku.edu.mie-u.ac.jp/~okumura/stat/fishertest.html
  cor.test(df1b$咳 , df1b$発熱)
  count_a = df1b %>% group_by(咳, 発熱) %>% summarize(n = n())
  count_a
  fisher.test(matrix(count_a$n , nrow = 2))
  cor.test(df1b$味覚 , df1b$嗅覚)
  df1b %>% group_by(味覚, 嗅覚) %>% summarize(n = n())
  count_b = df1b %>% group_by(味覚, 嗅覚) %>% summarize(n = n())
  fisher.test(matrix(count_b$n , nrow = 2))
  
  us
  fisher.test(matrix(df1b %>% group_by(味覚, 発熱) %>% summarize(n = n()) %>% pull(n) , nrow = 2))
  fisher.test(matrix(df1b %>% group_by(嗅覚, 発熱) %>% summarize(n = n()) %>% pull(n) , nrow = 2))
  fisher.test(matrix(df1b %>% group_by(頭痛, 発熱) %>% summarize(n = n()) %>% pull(n) , nrow = 2))
  fisher.test(matrix(df1b %>% group_by(咽頭痛, 発熱) %>% summarize(n = n()) %>% pull(n) , nrow = 2))



}
sf2 = sf_all %>% spread(key = run_sym , value = n , fill = 0 )  %>%
  rename(with = "TRUE"  , without = "FALSE")
ordered_sym = sf2 %>% group_by(symptom) %>% summarize(total = sum(with)) %>% ungroup %>% arrange(desc(total))%>% pull(symptom)
sf2 = sf2 %>% mutate(symptom = factor(symptom , ordered_sym))

sf3 =
library(tidyverse)
library(ggrepel)

windowsFonts("YuGo" = windowsFont("游ゴシック"))
#windowsFonts("YuGo" = windowsFont("HG創英角ゴシックUB"))
windowsFonts("POP"=  windowsFont("HGPSoeiKakupoptai"))
theme_set(theme_minimal(base_family = "POP"))
theme_set(theme_minimal(base_family = "YuGo"))


ggplot(df1 %>% group_by(年代) %>% summarize(n = n()), aes(x = 年代, y = n , 
                                                        fill = 年代, 
                                                        color = 年代 )) +
  geom_bar(stat = "identity") + guides(color = F, fill = F)

ggplot(df1 %>% group_by(年代, is_na_biko) %>% summarize(n = n()) %>% 
         spread(key = is_na_biko , value = n , fill = 0) %>% 
         rename("備考なし" = "TRUE" , "備考あり" = "FALSE")
         , aes(x = 備考なし, y = 備考あり   , label = 年代 , color = 年代) ) +
  geom_point() + geom_text_repel() + guides(color = F, fill = F) + 
  geom_smooth(method = "lm" , se = F)

ggplot(df1 %>% group_by(年代, is_na_biko) %>% summarize(n = n()), aes(x = 年代, y = n , 
                                                        fill = 年代, 
                                                        color = 年代 )) +
  geom_bar(stat = "identity") + guides(color = F, fill = F) + 
  labs(x = NULL , y = "人数") +
  theme(axis.text.x = element_text(angle = 90)) +
  facet_wrap(~is_na_biko , labeller = labeller(is_na_biko = c("TRUE" = "備考なし" , "FALSE" = "備考あり")))



sf2_age  =sf2 %>% group_by(年代, symptom) %>% summarize(with = sum(with) , without = sum(without)) %>% ungroup
ggplot(sf2_age ,  aes(y = 年代, x = with / (with + without) , 
                  fill = 年代, 
                  color = 年代 )) + 
  geom_bar(stat = "identity") + 
  scale_x_continuous(labels = scales::percent , name = "比率") + 
  #coord_flip() +
  labs(y = NULL) +
  theme(axis.text.x = element_text(angle = 90)) +
  facet_wrap(~symptom) + guides(color = F, fill = F)



ggplot(sf2_age%>% filter(symptom %in% symptoms[1:12]) ,  aes(y = 年代, x = with / (with + without) , 
                  fill = 年代, 
                  color = 年代 )) + 
  geom_bar(stat = "identity") + 
  scale_x_continuous(labels = scales::percent , name = "比率") + 
  #coord_flip() +
  labs(y = NULL) +
  theme(axis.text.x = element_text(angle = 90)) +
  facet_wrap(~symptom , nrow = 2) + guides(color = F, fill = F)



sf2_age_na_biko  =sf2 %>% filter(is_na_biko)%>% group_by(年代, symptom ) %>% summarize(with = sum(with) , without = sum(without)) %>% ungroup
ggplot(sf2_age_na_biko ,  aes(y = 年代, x = with / (with + without) , 
                  fill = 年代, 
                  color = 年代 )) + 
  geom_bar(stat = "identity") + 
  scale_x_continuous(labels = scales::percent , name = "比率") + 
  #coord_flip() +
  labs(y = NULL) +
  theme(axis.text.x = element_text(angle = 90)) +
  facet_wrap(~symptom) + guides(color = F, fill = F)

sf2_age_with_biko  =sf2 %>% filter(!is_na_biko)%>% group_by(年代, symptom ) %>% summarize(with = sum(with) , without = sum(without)) %>% ungroup
ggplot(sf2_age_with_biko ,  aes(y = 年代, x = with / (with + without) , 
                  fill = 年代, 
                  color = 年代 )) + 
  geom_bar(stat = "identity") + 
  scale_x_continuous(labels = scales::percent , name = "比率") + 
  #coord_flip() +
  labs(y = NULL) +
  theme(axis.text.x = element_text(angle = 90)) +
  facet_wrap(~symptom) + guides(color = F, fill = F)


sf2_age_biko  =sf2 %>% group_by(is_na_biko , 年代, symptom) %>% summarize(with = sum(with) , without = sum(without)) %>% ungroup

sf2_age_biko = sf2_age_biko  %>% mutate(ratio = with / (with + without))

wf2_age_biko  = sf2_age_biko %>% select(-starts_with("with"))%>% 
  spread(key = is_na_biko , value = ratio) %>%
  rename("備考あり" = "FALSE"  , "備考なし" = "TRUE")


ggplot(wf2_age_biko , aes(x = 備考なし ,y  = 備考あり , color = 年代 , label = 年代)) + 
  geom_point()　+ geom_text_repel(size = 2) + facet_wrap(~symptom) + guides(color = F, fill = F)


sf2_biko  =sf2 %>% group_by(is_na_biko , symptom) %>% summarize(with = sum(with) , without = sum(without)) %>% ungroup

sf2_biko = sf2_biko  %>% mutate(ratio = with / (with + without))

wf2_biko  = sf2_biko %>% select(-starts_with("with"))%>% 
  spread(key = is_na_biko , value = ratio) %>%
  rename("備考あり" = "FALSE"  , "備考なし" = "TRUE")


ggplot(wf2_biko , aes(x = 備考なし ,y  = 備考あり , label = symptom , color = symptom)) + 
  geom_point()　+ geom_text_repel(size = 2)  + guides(color = F, fill = F) + geom_abline(alpha = 0.5)



# delete below 